In [1]:
from utils.apps import SimpleApp
from langchain_openai import ChatOpenAI
from utils.db import get_database_schema_execute_all
import os
from keys import OPENAI_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_KEY  

# 1 | TEST - DB: test1

In [2]:
# Change path to csv files and prompt
path_to_csv_files = 'testDBs/test1/db/'
path_to_csv_schema_file = 'testDBs/test1/csv_schema.xlsx'
path_to_groud_truth_eventlog = 'testDBs/test1/ground-truth-eventlog.csv'
db_output_dir = 'example.db'
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #gpt-4, gpt-3.5-turbo
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)
prompt=f"""Consider the following db schema:
            {db_schema}
            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notion, combine the necessary tables to obtain this case notion.
            
            Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. 
            In activity_id leave out "_created".
            Use quotes for identifiers. Provide only the query."""


CREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" DATETIME)
CREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"))
CREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
Data inserted into table order
Data inserted into table invoice
Data inserted into table payment
Data inserted into table shipment
Database created: example.db


In [3]:
print(prompt)

Consider the following db schema:
            TABLE invoice:
CREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));

TABLE order:
CREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" DATETIME);

TABLE payment:
CREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));

TABLE shipment:
CREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" DATETIME, FOREIGN KEY ("order_id") REFERENCES "order"("id"));

            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notio

In [8]:
AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)
print('\n EXECUTED SQL STATEMNENT: \n', result['agent_response'])

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers



 EXECUTED SQL STATEMNENT: 
 SELECT "order"."id" AS case_id, 'order' AS activity_id, "order"."creation_date" AS timestamp
FROM "order"
UNION
SELECT "invoice"."order_id" AS case_id, 'invoice' AS activity_id, "invoice"."creation_date" AS timestamp
FROM "invoice"
UNION
SELECT "invoice"."order_id" AS case_id, 'payment' AS activity_id, "payment"."creation_date" AS timestamp
FROM "invoice"
JOIN "payment" ON "invoice"."id" = "payment"."invoice_id"
UNION
SELECT "shipment"."order_id" AS case_id, 'shipment' AS activity_id, "shipment"."creation_date" AS timestamp
FROM "shipment";


In [9]:
result['sqlexecuter']

,case_id,activity_id,timestamp
0,o1,invoice,2017-08-15 09:13:27
1,o1,order,2017-08-11 10:33:37
2,o1,payment,2017-08-21 16:26:13
3,o1,shipment,2017-08-24 11:36:35
4,o2,invoice,2017-08-17 17:38:36
5,o2,order,2017-08-13 16:18:15
6,o3,invoice,2017-08-19 23:03:45
7,o3,order,2017-08-15 22:02:53
8,o3,payment,2017-08-23 18:42:09
9,o4,invoice,2017-08-21 14:25:04


In [10]:
result['result']

{'prec': 0.0, 'rec': 0.0, 'f1': 0}

# 2 | TEST - DB: UWV_case_1

In [11]:
# Change path to csv files and prompt
path_to_csv_files = 'testDBs/UWV_case_1/db/'
path_to_csv_schema_file = 'testDBs/UWV_case_1/csv_schema.xlsx'
path_to_groud_truth_eventlog = 'testDBs/UWV_case_1/AutoDrive_Result_20201515_Sample.csv'
db_output_dir = 'db_UWV_case_1.db'
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #gpt-4, gpt-3.5-turbo
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)
prompt=f"""Consider the following db schema:
            {db_schema}
            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notion, combine the necessary tables to obtain this case notion.
            
            Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. 
            In activity_id leave out "_created".
            Use quotes for identifiers. Provide only the query."""

CREATE TABLE "AutoDrive_Event_300_20201215_Sample" ("id" PRIMARY KEY, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_301" TEXT)
CREATE TABLE "AutoDrive_Event_400_20201215_Sample" ("id" PRIMARY KEY, "EventDateTime" DATETIME)
CREATE TABLE "AutoDrive_Event_500_20201215_Sample" ("session_id" TEXT, "id" PRIMARY KEY, "ActivityID_501" DATETIME, "ActivityID_502" DATETIME, "ActivityID_503" DATETIME, "ActivityID_504" DATETIME, "ActivityID_505" DATETIME, "ActivityID_506" DATETIME, "ActivityID_507" DATETIME, "Attribute_510" TEXT)
CREATE TABLE "AutoDrive_Event_100_20201215_Sample" ("id" PRIMARY KEY, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_101" TEXT)
CREATE TABLE "AutoDrive_Event_200_20201215_Sample" ("id" PRIMARY KEY, "EventDateTime" DATETIME, "ActivityID" TEXT)
Data inserted into table AutoDrive_Event_300_20201215_Sample
Data inserted into table AutoDrive_Event_400_20201215_Sample
Data inserted into table AutoDrive_Event_500_20201215_Sample
Data inserted into table AutoDri

In [13]:
AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)
print('\n EXECUTED SQL STATEMNENT: \n', result['agent_response'])

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers



 EXECUTED SQL STATEMNENT: 
 SELECT id AS case_id, 'ActivityID' AS activity_id, EventDateTime AS timestamp
FROM AutoDrive_Event_100_20201215_Sample
UNION ALL
SELECT id, 'ActivityID', EventDateTime
FROM AutoDrive_Event_200_20201215_Sample
UNION ALL
SELECT id, 'ActivityID', EventDateTime
FROM AutoDrive_Event_300_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_501', ActivityID_501
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_502', ActivityID_502
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_503', ActivityID_503
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_504', ActivityID_504
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_505', ActivityID_505
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_506', ActivityID_506
FROM AutoDrive_Event_500_20201215_Sample
UNION ALL
SELECT id, 'ActivityID_507', ActivityID_507
FROM AutoDrive_Event_500_20201215_Sample;


In [14]:
result['sqlexecuter']

,case_id,activity_id,timestamp
0,16969436,ActivityID,08:53.8
1,16981611,ActivityID,32:07.1
2,16982614,ActivityID,41:47.4
3,16983477,ActivityID,42:10.8
4,16969436,ActivityID,01:00.0
5,16981611,ActivityID,50:00.0
6,16982614,ActivityID,32:00.0
7,16983477,ActivityID,24:00.0
8,16969436,ActivityID,2019-12-30
9,16981611,ActivityID,2019-12-05


In [15]:
result['result']

"Can't calculate Precision, Recall and F1. DataFrames must have the same columns"